In [1]:
import pandas as pd
import xarray as xr
import numpy as np

In [ ]:
site_df = pd.read_csv('../data/DeDe_48stations/DeDe_48sites_metadata.csv')
ssrd_ds = xr.open_dataset('../data/ERA5/radiation/era_ssrd_2023.nc')
str_ds = xr.open_dataset('../data/ERA5/radiation/STR.nc')
ssr_ds = xr.open_dataset('../data/ERA5/radiation/SSR.nc')

In [19]:
for i, row in site_df.iterrows():

    ssrd_df = ssrd_ds.sel(latitude=row['lat'],longitude=row['lng'], method='nearest').to_dataframe()
    str_df = str_ds.sel(latitude=row['lat'],longitude=row['lng'], method='nearest').to_dataframe()
    ssr_df = ssr_ds.sel(latitude=row['lat'],longitude=row['lng'], method='nearest').to_dataframe()

    df = pd.concat([ssrd_df['ssrd'], str_df['str'], ssr_df['ssr']], axis=1)
    df = df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    df = df[df.index.year == 2023]
    df_1h = df.reset_index()
    df_1h.rename(columns={'valid_time': 'Datetime'}, inplace=True)

    df_1h.to_csv(f'../data/ERA5/radiation/all_site_1hr/era5_1hr_2023_site{row['Site_id']:02d}.csv', index=False)

    df = df.resample('15min').asfreq()
    df['ssrd'].interpolate(method='linear', inplace=True)
    df['str'].interpolate(method='linear', inplace=True)
    df['ssr'].interpolate(method='linear', inplace=True)
    df.reset_index(inplace=True)
    df.rename(columns={'valid_time': 'Datetime'}, inplace=True)

    df.to_csv(f'../data/ERA5/radiation/all_site_15min/era5_15min_2023_site{row['Site_id']:02d}.csv', index=False)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_17284\4058692455.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ssrd'].interpolate(method='linear', inplace=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_17284\4058692455.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.